In [1]:
%load_ext autoreload
%autoreload 3

In [2]:
import logging
import sys

import ferc1_transform_refactor as tfr
import pandas as pd

In [3]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(stream=sys.stdout)
formatter = logging.Formatter("%(message)s")
handler.setFormatter(formatter)
logger.handlers = [handler]

In [4]:
dataset_params = tfr.TRANSFORM_PARAMS
dataset_params

{'test_table_id': {'rename_columns': {'columns': {'netgen': 'net_generation_kwh',
    'heat_content': 'fuel_btu_per_unit',
    'units': 'fuel_units',
    'fuel_type': 'fuel_type'}},
  'convert_units': {'net_generation_kwh': {'multiplier': 0.001,
    'pattern': '(.*)_kwh$',
    'repl': '\\1_mwh'},
   'fuel_btu_per_unit': {'multiplier': 1e-06,
    'pattern': '(.*)_btu(.*)$',
    'repl': '\\1_mmbtu\\2'}},
  'categorize_strings': {'fuel_type': {'categories': {'coal': {'black',
      'dinosaur',
      'goo',
      'xenoforming'},
     'oil': {'astroglide', 'fish', 'olive', 'sunflower'},
     'gas': {'cow farts', 'elon musk', 'unicorn farts'}}}}}}

In [5]:
table_params = tfr.TableTransformParams.parse_obj(dataset_params["test_table_id"])
table_params

TableTransformParams(rename_columns=RenameColumns(columns={'netgen': 'net_generation_kwh', 'heat_content': 'fuel_btu_per_unit', 'units': 'fuel_units', 'fuel_type': 'fuel_type'}), convert_units={'net_generation_kwh': UnitConversion(multiplier=0.001, adder=0.0, pattern=re.compile('(.*)_kwh$'), repl='\\1_mwh'), 'fuel_btu_per_unit': UnitConversion(multiplier=1e-06, adder=0.0, pattern=re.compile('(.*)_btu(.*)$'), repl='\\1_mmbtu\\2')}, categorize_strings={'fuel_type': StringCategories(categories={'coal': {'goo', 'black', 'xenoforming', 'dinosaur'}, 'oil': {'olive', 'fish', 'sunflower', 'astroglide'}, 'gas': {'cow farts', 'unicorn farts', 'elon musk'}})})

In [6]:
table_params.rename_columns

RenameColumns(columns={'netgen': 'net_generation_kwh', 'heat_content': 'fuel_btu_per_unit', 'units': 'fuel_units', 'fuel_type': 'fuel_type'})

In [7]:
tttfr = tfr.TestTableTransformer("test_table_id")
actual = tttfr.apply({"test_table_id": tfr.TEST_INPUT_DF})["test_table_id"]

actual

,fuel_units,fuel_type,net_generation_mwh,fuel_mmbtu_per_unit
0,tons,💩,1.0,17.0
1,barrels,oil,2.0,5.8
2,mcf,gas,3.0,0.1


In [8]:
tfr.EXPECTED_OUTPUT_DF

,net_generation_mwh,fuel_mmbtu_per_unit,fuel_units,fuel_type
0,1.0,17.0,tons,💩
1,2.0,5.8,barrels,oil
2,3.0,0.1,mcf,gas


In [9]:
pd.testing.assert_frame_equal(
    actual.sort_index(axis="columns"),
    tfr.EXPECTED_OUTPUT_DF.sort_index(axis="columns")
)